In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [17]:
# Generaset Dataset Dummy
np.random.seed(42)
n_samples = 5000

umur_bangunan = np.random.randint(1, 100, size=n_samples)
luas_bangunan = np.random.randint(50, 500, size=n_samples)
tipe_material = np.random.choice([0, 1], size=n_samples)  # 0 = standar, 1 = tahan api
lokasi_risiko = np.random.choice([0, 1], size=n_samples)  # 0 = aman, 1 = rawan bencana
nilai_properti = np.random.uniform(100_000_000, 5_000_000_000, size=n_samples).astype(int)

In [18]:
# Membuat Label Risiko dan Premi
risk_score = (
    (umur_bangunan > 50).astype(int) +
    (luas_bangunan > 300).astype(int) +
    (tipe_material == 0).astype(int) +
    lokasi_risiko
)

risk_label = pd.cut(risk_score, bins=[-1, 1, 2, 4], labels=[0, 1, 2]).astype(int)

base_premi = 200_000
premi = base_premi + (risk_label * 150_000) + (nilai_properti * 0.005)
premi = premi.astype(int)

In [19]:
#  Membuat DataFrame dan Membagi Data
df = pd.DataFrame({
    'umur_bangunan': umur_bangunan,
    'luas_bangunan': luas_bangunan,
    'tipe_material': tipe_material,
    'lokasi_risiko': lokasi_risiko,
    'nilai_properti': nilai_properti,
    'risk_label': risk_label,
    'premi': premi
})

features = df[['umur_bangunan', 'luas_bangunan', 'tipe_material', 'lokasi_risiko', 'nilai_properti']]
target_risk = df['risk_label']
target_premi = df['premi']

X_train, X_test, y_risk_train, y_risk_test, y_premi_train, y_premi_test = train_test_split(
    features, target_risk, target_premi, test_size=0.2, random_state=42
)

In [20]:
# Standarisasi Fitur Numerik
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

X_train_scaled[['umur_bangunan', 'luas_bangunan', 'nilai_properti']] = scaler.fit_transform(
    X_train[['umur_bangunan', 'luas_bangunan', 'nilai_properti']]
)
X_test_scaled[['umur_bangunan', 'luas_bangunan', 'nilai_properti']] = scaler.transform(
    X_test[['umur_bangunan', 'luas_bangunan', 'nilai_properti']]
)

In [21]:
# Membangun Model TensorFlow
input_layer = tf.keras.Input(shape=(5,))
x = tf.keras.layers.Dense(64, activation='relu')(input_layer)
x = tf.keras.layers.Dense(32, activation='relu')(x)

risk_output = tf.keras.layers.Dense(3, activation='softmax', name='risk_label')(x)
premi_output = tf.keras.layers.Dense(1, activation='linear', name='premi')(x)

model = tf.keras.Model(inputs=input_layer, outputs=[risk_output, premi_output])
model.compile(
    optimizer='adam',
    loss={'risk_label': 'sparse_categorical_crossentropy', 'premi': 'mse'},
    metrics={'risk_label': 'accuracy', 'premi': 'mae'}
)

In [22]:
# Melatih Model
history = model.fit(
    X_train_scaled,
    {'risk_label': y_risk_train, 'premi': y_premi_train},
    validation_data=(X_test_scaled, {'risk_label': y_risk_test, 'premi': y_premi_test}),
    epochs=30,
    batch_size=32,
    verbose=2
)

Epoch 1/30
125/125 - 6s - 49ms/step - loss: 221999662104576.0000 - premi_loss: 221999662104576.0000 - premi_mae: 13068815.0000 - risk_label_accuracy: 0.3602 - risk_label_loss: 1.1817 - val_loss: 219090660622336.0000 - val_premi_loss: 219267911909376.0000 - val_premi_mae: 12985436.0000 - val_risk_label_accuracy: 0.3270 - val_risk_label_loss: 1.1406
Epoch 2/30
125/125 - 1s - 8ms/step - loss: 221995987894272.0000 - premi_loss: 221995987894272.0000 - premi_mae: 13068699.0000 - risk_label_accuracy: 0.3225 - risk_label_loss: 1.2151 - val_loss: 219082875994112.0000 - val_premi_loss: 219260076949504.0000 - val_premi_mae: 12985181.0000 - val_risk_label_accuracy: 0.2750 - val_risk_label_loss: 1.2279
Epoch 3/30
125/125 - 1s - 8ms/step - loss: 221980787736576.0000 - premi_loss: 221980787736576.0000 - premi_mae: 13068211.0000 - risk_label_accuracy: 0.3205 - risk_label_loss: 1.3109 - val_loss: 219059455000576.0000 - val_premi_loss: 219236454629376.0000 - val_premi_mae: 12984439.0000 - val_risk_label

In [23]:
# Evaluasi Model
eval_results = model.evaluate(X_test_scaled, {'risk_label': y_risk_test, 'premi': y_premi_test}, verbose=2)
print(f"Test Loss dan Metrics: {eval_results}")

32/32 - 0s - 9ms/step - loss: 199359463424000.0000 - premi_loss: 199381877784576.0000 - premi_mae: 12361196.0000 - risk_label_accuracy: 0.3030 - risk_label_loss: 23.1782
Test Loss dan Metrics: [199359463424000.0, 23.17821502685547, 199381877784576.0, 12361196.0, 0.30300000309944153]


In [24]:
# Menyimpan Model
model.save('model_asuransi_properti.h5')

import joblib
joblib.dump(scaler, 'scaler_properti.pkl')

['scaler_properti.pkl']

In [25]:
from tensorflow.keras.models import load_model
import joblib

model = load_model('model_asuransi_properti.h5', compile=False)
scaler = joblib.load('scaler_properti.pkl')

def prediksi_risiko_premi_properti(umur_bangunan, luas_bangunan, tipe_material, lokasi_risiko, nilai_properti, base_premi=200_000):
    input_df = pd.DataFrame({
        'umur_bangunan': [umur_bangunan],
        'luas_bangunan': [luas_bangunan],
        'tipe_material': [1 if tipe_material == "tahan api" else 0],
        'lokasi_risiko': [1 if lokasi_risiko == "rawan" else 0],
        'nilai_properti': [nilai_properti]
    })

    input_scaled = input_df.copy()
    input_scaled[['umur_bangunan','luas_bangunan','nilai_properti']] = scaler.transform(
        input_df[['umur_bangunan','luas_bangunan','nilai_properti']]
    )

    pred = model.predict(input_scaled)
    kelas_risiko = np.argmax(pred[0], axis=1)[0]
    premi_final = base_premi + (kelas_risiko * 150_000) + (nilai_properti * 0.005)
    return int(kelas_risiko), int(premi_final)

# Contoh penggunaan
risk, premi = prediksi_risiko_premi_properti(umur_bangunan=30, luas_bangunan=200, tipe_material='standar', lokasi_risiko='aman', nilai_properti=1_000_000_000)
print(f"Kelas Risiko: {risk}")
print(f"Premi: Rp {premi}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Kelas Risiko: 2
Premi: Rp 5500000
